このページは以下のリンクより， google colaboratoryから動作させることができる．
- [Open with Colab](https://colab.research.google.com/github/crotsu/Bousai_AI/blob/master/chap3_NeuralNetwork/NeuralNetwork_document.ipynb)

# ニューラルネットワーク

## 1. はじめに
ニューラルネットワークは，脳の神経回路網をモデル化した機械学習の手法である．  
近年，深層学習（Deep Learning）の登場により再びニューラルネットワークが注目を集めている．  
深層学習は，画像認識，音声認識，自然言語処理など様々な分野で取り入られており，Google社をはじめ， 多くの企業で利用，研究がされている．  
本資料では，ニューラルネットワークの基本的な原理， 学習方法について記述する．  

## 2. 基本構造
ニューラルネットワークは，複数のニューロンとニューロン同士を結合するシナプスから構成される.  
シナプスは各結合の強さを表す結合重みを持っている.  
ニューラルネットワークの図を示す.  

<img src="img/neuralnetwork.png" width="400">
<div style="text-align: center;">
図1.  3層の階層型ニューラルネットワーク
</div>

ニューラルネットワークは，とても複雑な関数だと思うと理解が早い．  
関数なので，入力$x$と，その関数$f(x)$の出力$y$をとる．  
さらに，この関数$f(x)$は，合成関数となっており，イメージとしては，$f(x) = u(s(g(x)+h(x)) + t(m(x)+n(x)) )$のように複雑な合成関数である．  
各関数にはパラメータがあり，これをうまく調整すると，ある入力$x$に対して，所望の$y$が出力するようになる．  
この調整をトレーニングデータから自動的に行うのがニューラルネットワークである．  

<img src="img/system.png" width="400">
<div style="text-align: center;">
図2.  ニューラルネットワークのイメージ
</div>

ニューラルネットワークは，入力層，中間層，出力層の多層構造になっており，各ニューロンは前の層のニューロンの出力を入力として受け取り，計算を行ったあと，次のニューロンへと出力する．  
各ニューロンの出力が次々と伝播していき，出力層のニューロンからネットワークの出力を得る．  
単体のニューロンの構造を図に示す．  

<img src="img/neuron.png" width="400">
<div style="text-align: center;">
図3.   単体のニューロン
</div>

ニューロンは複数の入力を受け取り，1つの値を出力する．  
ある層の上位ニューロン$j$は，その前の層の$I$個の下位ニューロン出力$o_i$を入力として受け取る．  
このとき，ニューロン間の重み$w_ij$と入力$o_i$の積和計算する．  
そして，閾値$\theta_j$を加えた値を重み付き和$X_j$として算出する．  
その重み付き和の値を活性化関数$f(x)$の引数として，活性化関数の値をニューロン$j$の出力とする．  
これらを式で表すと以下のようになる．  

$$
X_j =  \sum_{i=1}^{I} w_{ij} o_i + \theta_j \\
o_j =  f(X_j)
$$

活性化関数には，以下のシグモイド関数や双曲線正接関数などが使われる．

$$
f(x) = \frac{1}{1 + \exp(-\epsilon x)} \\
f(x) = \tanh(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}
$$

<img src="img/sigmoid.png" width="400">
<div style="text-align: center;">
図4.   シグモイド関数と双曲線正接関数のグラフ
</div>

ここで$\epsilon$はシグモイド関数の$x=0$のときの傾きを表している．  
$\epsilon$が小さいほど傾きが緩やかになり，大きいほど傾きが傾きが大きくなる．  
傾きが急なシグモイド関数ほど，少しの変化で0あるいは1の出力となるので，反応が敏感な関数であると言える．

<img src="img/sigmoid2.png" width="400">
<div style="text-align: center;">
図5.   シグモイド関数と双曲線正接関数のグラフ
</div>

近年，これらの活性化関数の代わりに正規化線形関数(Rectified linear function)が利用されることが多くなっている．  
一般的には，ReLUと呼ばれる．  
ReLUは，

- $\max(0,x)$は単純ゆえに早い
- 0を作るので，スパース性につながる
- $x>0$の部分では微分値が常に1であるため勾配消失の心配はない  

という利点を持つ．  
ReLUの式とグラフを以下に示す．  

$$
f(x) = \max(0,x)
$$

<img src="img/relu.png" width="400">
<div style="text-align: center;">
図6.   ReLU関数
</div>

以上の計算を入力層側のニューロンから順に行う．  
ただし，入力層のニューロンはこれらの計算を行わず，入力をそのまま出力する．  
つまり，入力層2個，中間層2個，出力層1のネットワークは，厳密は以下の図のようになる．  

<img src="img/ff1.png" width="400">
<div style="text-align: center;">
図7.  本来，入力層にはニューロンはない
</div>

しかし，イラストのバランスが悪いので，入力ニューロンは描いてネットワークを表現することが多い．

<img src="img/ff2.png" width="400">
<div style="text-align: center;">
図8.  バランスが悪いので入力ニューロンは描くことが多い
</div>

この前向きの計算により，入力層への入力に対し，中間層を経て，出力層から出力を得ることができる．

## 3. 学習の目的

ニューラルネットワークは，理想の出力に近いほど性能が高いといえる．  
各出力ニューロン$k$に対応する教師信号$t_k$が与えられたとき，各入力パターンに対する誤差$E_p$とP個の入力パターンの誤差の総和$E_{all}$は以下の式で表すことができる．

$$
E_p = \frac{1}{2}\sum_{k = 1}^{K}(o_k - t_k)^2 \\
E_{all}  =  \sum_{p = 1}^{P} E_p
$$

ニューラルネットワークの学習は，この誤差総和$E_{all}$を最小化し，入力に対して適切な出力が得られるように各結合重みと閾値を調整することである．

## 4. 仮想ニューロン

ニューラルネットワークの出力を最適化するには，各結合重みと閾値を調整すれば良いが，それぞれを修正するのは手間がかかる．  
そこで閾値を重みとして扱うために仮想ニューロンを導入する．  
入力層，中間層，出力層のニューロン数がそれぞれ$I$個，$J$個，$K$個のニューラルネットワークに仮想ニューロンを導入した形を図に示す．  

<img src="img/virtual.png" width="400">
<div style="text-align: center;">
図9.  仮想ニューロン
</div>

仮想ニューロンは出力層を除く，各層のニューロンの最後に追加する．  
図のニューラルネットワークでは，$I + 1$番目，$J + 1$番目に追加している．  
各仮想ニューロンは次の層の全ニューロンと結合しており，結合先のニューロンの閾値を結合重みとしている．  
また，各仮想ニューロンは常に$1$を出力している．これにより，重み付け和の式は以下のように置き換えられる．  

$$
\begin{eqnarray}
	X_j & = & \sum_{i = 1}^{I} w_{ij} o_i - \theta_j \nonumber \\
	& = & \sum_{i = 1}^{I} w_{ij} o_i + w_{(I + 1), j} \nonumber \\
	& = & \sum_{i = 1}^{I} w_{ij} o_i + w_{(I + 1), j} \cdot 1 \nonumber \\
	& = & \sum_{i = 1}^{I} w_{ij} o_i + w_{(I + 1), j} \cdot o_{I + 1} \nonumber \\
	& = & \sum_{i = 1}^{I + 1} w_{ij} o_i
	\end{eqnarray}
$$

この式変形により，閾値も重みとして扱うことができる．  
これにより，この後のの重みを修正する式によって，間接的に閾値も修正することができる．  
ただし，閾値がなくなったわけではないので，実装のときには注意が必要である．

## 5. 非線形問題と線形問題

### 5.1 問題の困難度

前向き計算の例を考えてみよう．  
例えば，魚Aと魚Bを仕分けするシステムを作りたいとする．  
ベルトコンベアの乗って流れてきた魚の体長と体重を測って，それらから魚を推定する．  
体長をx, 体重をyとすると2次元平面上にマッピングできる．  
図の左のようになる．  
これを見ると，青の魚Aと赤の魚Bが入り組んでいて，分類が難しそうである．  

一方，簡単な例として図の右のようなものを考えてみよう．  
この図では，きれいに分けることができそうである．  


<img src="img/fish1.png" width="800">
<div style="text-align: center;">
図10.  魚の分類
</div>

なぜ，左側の図の方が難しいのだろうか．  
それは，以下の図のように，右側の問題は直線で分離が可能な線形問題だからである．  
線形分離とは，元の次元$n$から1次引いた$n-1$次元の超平面で分離できるかどうかである．  
今，2次元（体長と体重）の特徴ベクトルなので2次元平面を分離する超平面は1次元の直線である．  
一方，例えば3次元のデータを分離するのは，2次元の平面ということになる．  
同様に，$n$次元のデータを分離するのは，$n-1$次元の超平面ということがいえる．  
つまり，何次元のデータであっても超平面で分離できれば線形分離可能な問題ということになる．  

<img src="img/fish2.png" width="800">
<div style="text-align: center;">
図11.  魚の分類
</div>

魚の分類（難しい）の問題は非線形問題である．  
非線形問題で一番簡単な問題はXOR（排他的論理和）の問題である．  
XORは4つのデータしかないが，これを1次元の直線で分離させることはできない．  
そこで，ここでは，まず非線形問題で一番簡単なXORを学習させてみよう．  
逆に言えば，XORが学習できれば，本質が同じ魚の分類もできる．

<img src="img/fish3.png" width="800">
<div style="text-align: center;">
図12.  魚の分類
</div>

### 5.2 XORを学習したニューラルネットワークの前向き計算

 以下の図は，すでにXORを学習済みの重みと閾値のニューラルネットワークである．  
 例えば，(0,1)を入力すると，どうなるか計算してみよう．  
 正しく，1と出力されるはずである．  
 では，他の3パターンはどうだろうか？  
 いちいち手計算するのは大変なので，プログラムで動かしてみよう．  

<img src="img/xor1.png" width="800">
<div style="text-align: center;">
図13.  XORの例
</div>

## 6. デモ（前向き計算）

プログラムで動かすために，設計が必要であるが，ニューラルネットワークを理解をすることを最優先して，  
「拡張性ゼロのアホアホプログラム」  
で体験することにしよう．  
（配列や複雑な関数で構成されたプログラムは，ややこしくて本質が見えなくなります）  

では，どうするか？
各重みは，全部変数にしてしまおう！！！  
（なんと乱暴な！しかし，とりあえず理解するには，これが一番わかりやすい！）  

<img src="img/xor2.png" width="500">
<div style="text-align: center;">
図14.  XORの例
</div>

ニューロンAとニューロンB間の重みはwab  
ニューロンAの閾値はtha  
のように変数で表現します．  
（wはweight，thはthresholdの頭文字です）

[前向き計算（拡張性ゼロのアホアホプログラム）](./chap3_1_forward.ipynb)

### 演習
このプログラムのコメントアウト部分である，ランダムに重みと閾値を与える部分を実行すると，  
未学習なのでXORの出力にはならないことを確認してみよう．  
ニューラルネットワークは，初期重みと閾値をランダムに与えて，学習によって最適化する．  

### ニューラルネットワークは，でっかい複雑な「ただの関数」

上記のパラメータを展開すると，2入力1出力の関数になる．  
数学の関数$y=f(x_1, x_2)$と同じである．  
したがって，何か2入力を与えると，何か出力される．  
<img src="img/expand1.png" width="500">
<div style="text-align: center;">
図15.  2入力1出力のニューラルネットワーク
</div>

これを入力に対して，適切な出力がなされるように，パラメータ（重みと閾値）を求める．  
ニューラルネットワークは，ただの関数だということを理解することが大事である．

<img src="img/expand2.png" width="500">
<div style="text-align: center;">
図16.  2入力1出力の「ただの関数」
</div>

ちなみに，愚直にプログラムを数式にすると以下のようになる．  

$$
g(x) = \frac{1}{1+\exp(-4.0x))}
$$

$$
y = g(1.94 \times g(-1.54 x_1 + 1.60 x_2 - 0.92) -1.88 \times g(-1.21 x_1 + 1.29 x_2 + 0.58) + 0.88)\\
$$

[式を展開したプログラム](./chap3_2_expand.ipynb)

## 7. バックプロパゲーション法

最急降下法を用いて図のようにニューラルネットワークの出力層から入力層に向かって重みの調整を行う手法を誤差逆伝播法（Back Propagation）という．

<img src="img/bp1.png" width="500">
<div style="text-align: center;">
図17.  BP法の修正の方向
</div>

学習の目的は誤差総和$E_{all}$を最小化することだが$E_{all}$を最小化することは困難であるため，  
各パターンの誤差$E_p$が最小となるように重みを更新する．  
重みが変化すると出力と誤差も変化するため，  
$E_p$は$w$を引数とした関数$E_p(w)$と考えることができる．  
したがって，  $\frac{\partial E_p(w)}{\partial w}$を求めて，$w$を更新することで$E_p$を小さくすることができる．  

最急降下法を用いた重み$w$の更新式は以下のようになる．  	
$$
w_{t+1} = w_t - \eta \frac{\partial E_p(w_t)}{\partial w_t}
$$

この最急降下法を用いて各重みを出力層から入力層に向かって順に更新していく．  

<img src="img/bp2.png" width="500">
<div style="text-align: center;">
図18.  BP法導出するための記号
</div>

最初に中間層-出力層の任意の重み$w_{jk}$について考える．  
$E_p$の式には直接$w_{jk}$は存在しないので合成関数の微分を考える．  
$\frac{\partial E_p}{\partial w_{jk}}$の合成関数の微分は以下のようになる．

$$
\frac{\partial E_p}{\partial w_{jk}} = \frac{\partial E_p}{\partial o_k}
\frac{\partial o_k}{\partial X_k} \frac{\partial X_k}{\partial w_{jk}}
$$

これらの偏微分をそれぞれ計算すると以下のようになる．

$$
\frac{\partial E_p}{\partial o_k}  =  \frac{1}{2}\{2(o_k - t_k)\}
		= o_k - t_k \\
\frac{\partial o_k}{\partial X_k}  =  f'(X_k) \\
\frac{\partial X_k}{\partial w_{jk}}  =  o_j 
$$
	
以上より，中間層-出力層の重みの更新式は以下のようになる．
	
$$
w_{jk} = w_{jk} - \eta (o_k - t_k) f'(X_k) o_j
$$
	
さらに出力ニューロンに関係する式を$\delta_k$としてまとめると，この式は次のように表せる．

$$
\delta_k = \frac{\partial E_p}{\partial o_k} \frac{\partial o_k}{\partial X_k} = (o_k - t_k)f'(X_k) \\
w_{jk} = w_{jk} - \eta \delta_k o_j
$$





次に中間層以下の重み$w_{ij}$について考える．  
中間層以下の重みの合成関数の微分は以下のようになる．

$$
\frac{\partial E_p}{\partial w_{ij}}  = \sum_{k = 1}^{K} \frac{\partial E_p}{\partial o_k} \frac{\partial o_k}{\partial X_k} \frac{\partial X_k}{\partial o_j} \frac{\partial o_j}{\partial X_j} \frac{\partial X_j}{\partial w_{ij}}
$$

この式では，注目ニューロンより上位の層で結合している全てのニューロンの誤差を考慮するために$\sum$を取っている．  
この式の$\frac{\partial E_p}{\partial o_k} \frac{\partial o_k}{\partial X_k}$は上で求めた$\delta_k$が代入できるので，  
それ以外の微分について考える．  
各偏微分は以下のように計算できる．

$$
\frac{\partial X_k}{\partial o_j} = w_{jk} \\
\frac{\partial o_j}{\partial X_j} = f'(X_j) \\
\frac{\partial X_j}{\partial w_{ij}} = o_i 
$$

以上より，中間層以下の重みの更新式は以下のようになる．

$$
\delta_j  =  \sum_{k = 1}^{K} \frac{\partial E_p}{\partial o_k}
\frac{\partial o_k}{\partial X_k} \frac{\partial X_k}{\partial o_j}
\frac{\partial o_j}{\partial X_j} = \sum_{k = 1}^{K} \delta_k w_{jk} f'(X_j) \\
w_{ij}  =  w_{ij} - \eta \delta_j o_i
$$

以後，同じように繰り返し$\delta$を求めていけば，全ての重みを修正することができる．  
活性化関数がシグモイド関数の場合，$f'(x)$は以下のようになる．

$$
f'(x) = \frac{\epsilon \exp(-\epsilon x)}{(1 + \exp(-\epsilon x))^2} \\
= \epsilon \cdot \frac{\exp(-\epsilon x)}{1 + \exp(-\epsilon x)} \cdot \frac{1}{1 + \exp(-\epsilon x)} \nonumber \\
= \epsilon \cdot \frac{1 + \exp(-\epsilon x) - 1}{1 + \exp(-\epsilon x)} \cdot \frac{1}{1 + \exp(-\epsilon x)} \nonumber \\
= \epsilon \cdot \Biggl(1 - \frac{1}{1 + \exp(-\epsilon x)}\Biggr) \cdot \frac{1}{1 + \exp(-\epsilon x)} \nonumber \\
= \epsilon (1 - f(x))f(x) \nonumber \\
$$

またf(X_k) = o_kより  

$$
f'(X_k) = \epsilon (1 - o_k)o_k
$$

と簡単化できる．  

このようにシグモイド関数の場合は，各ニューロンの出力から$f'(X_k)$を求めることができる．  
以上をまとめると，各重みと閾値の更新式は以下のようになる．

- 中間層-出力層
$$
\delta_k = (o_k - t_k) \epsilon (1 - o_k)o_k \\
w_{jk} = w_{jk} - \eta \delta_k o_j \\
\theta_{k}  =  \theta_{k}-\eta \delta_k
$$

- 中間層以下
$$
\delta_j = \sum_{k = 1}^{K} \delta_k w_{jk} \epsilon (1 - o_j) o_j \\
w_{ij} = w_{ij} - \eta \delta_j o_i \\
\theta_{j} = \theta_{j}-\eta \delta_j
$$


### 補足 （合成関数の偏微分）

$z=f(x,y)$が$x,y$で全微分可能であり，$x=x(t), y=y(t)$が$t$で微分可能ならば，  
合成関数$z=f(x(t), y(t))$は$t$微分可能であり，次の式が成り立つ．  

$$
\frac{dz}{dt}= \frac{\partial z}{\partial x}\frac{\partial x}{\partial t} + \frac{\partial z}{\partial y}\frac{\partial y}{\partial t}
$$

図でイメージすると，$z$は$x$と$y$の関数であり，$x$と$y$は$t$の関数である．  
したがって，$t$から$x,y$への線分と，$x$から$z$，$y$から$z$への線分が必要となる．

<img src="img/bp3.png" width="250">
<div style="text-align: center;">
図19.  合成関数の関係
</div>

## 8. デモ（バックプロパゲーション法）

先程と同じように拡張性がないプログラムでBP法で重みを学習してみよう．  

[バックプロパゲーション法（拡張性ゼロのアホアホプログラム）](./chap3_3_backpropagation.ipynb)

## 9. 行列・ベクトル表現でニューラルネットワークを考える

ニューロンの説明では以下のようなスカラーを用いた．  
$$
X_j =  \sum_{i=1}^{I} w_{ij} o_i + \theta_j \\
o_j =  f(X_j)
$$

これは，行列とベクトルを使うともっと簡単に表現することができる．  

まず，下の図でスカラー計算を行ってみよう．

<img src="img/vector1.png" width="500">
<div style="text-align: center;">
図20.  行列・ベクトル表現のための変数名
</div>

$$
X_b = (wbd\times od + wbe\times oe) + thb\\
ob = f(X_b)\\
$$

$$
X_c = (wcd\times od + wce\times oe) + thc\\
ob = f(X_b)
$$

では，重みを行列，出力，閾値をベクトルで表現してみよう．

$$
W^{(1)} = 
\begin{pmatrix}
    wbd & wbe\\
    wcd & wce
\end{pmatrix}
$$

$$
o^{(0)} =
\begin{pmatrix}
    od\\
    oe
\end{pmatrix}
$$

$$
th^{(1)} =
\begin{pmatrix}
    thb\\
    thc
\end{pmatrix}
$$

すると，行列とベクトルの積で簡単に表現することができる．

$$
X^{(1)}=
\begin{pmatrix}
    X_b\\
    X_c
\end{pmatrix}
$$

$$
o^{(0)} =
\begin{pmatrix}
    od\\
    oe
\end{pmatrix}
$$
とおけば，以下のようになる．

$$
X^{(1)} = W^{(1)}o^{(0)}+th^{(1)}\\
o^{(1)} = f(X^{(1)})
$$

Numpyは行列・ベクトル演算が簡単に行える．  
したがって，スカラー表現のプログラムを行列・ベクトル表現のプログラムに変更すれば拡張性が高いプログラムを作成できる．

## 10. デモ（行列・ベクトル表現）

Numpyを使って行列・ベクトル表現の拡張性の高いプログラムで試してみよう．

[行列・ベクトル表現のプログラム](./chap3_4_vector.ipynb)

## 11. フレームワークを使う(TensorFlow + Keras)

ここまで見てきたように，頑張れば自力でニューラルネットワークを実装することができる．  
しかし，とても大変だったことがわかると思う．  
さらに，今，Deep Learningの研究を行っている人が多数いる．  
その人達と成果物を共有しようとするとなると，各自が自作したプログラムでは議論が難しい．  

そこで，Deep Learningを動かす枠組み（フレームワーク）が多く開発されている．  
ここでは，それらの一つであるGoogleが開発したTensorFlowを紹介する．  
TensorFlowはニューラルネットワークを動かすというよりは，テンソル（3次元以上の多次元行列）の演算を行うために開発されたものである．  
もちろん，Deep Learningのために開発されたのであるが，ニューラルネットワークは行列の演算を多く行う．  
そのため，行列演算のために，しかも並列計算が可能になるように開発されたものがTensorFlowである．  

さらに，　KerasはTensorFlowのフロントエンドあり，Deep Learningのモデルを短い記述で実装できる．  
一般的に，TensorFlowでDeep Learningを実装しようとすると，Kerasを用いる．

## 12. デモ（TensorFlow + Keras）

TensorFlow + Kerasを使えば簡単に実装できる．  
ニューラルネットワークの構造を変えたり，活性化関数を変更したり，最適化のアルゴリズムを変更したり，とにかくいろいろできる．  
さらに，グラフを表示したり，ネットワークの出力を可視化したり，とにかく機能が豊富である．  

そのため，せっかく簡単に記述できるにも関わらず，とても複雑になり，初心者にはわかりにくいサンプルプログラムをが多い．  
そこで，下記のプログラムはこれまで説明してきたXORを，TensorFlow + Kerasで短く，わかりやすく記述することを目的とする．  

[TensorFlow+Keras](./chap3_5_keras.ipynb)